In [ ]:
import os
import pandas
import numpy

In [ ]:
def has_exceeded(channel_no, threshold, line2):
    highest = 0
    list1 = []
    for a in range(8):
        list1.append(float(line2[23+channel_no+a*19]))
    highest = max(list1)
#     print("the highest temp of channel number "+str(channel_no)+"is"+str(highest))
#     print("\n")
    if(highest>threshold):
        return 1
    else:
        return 0

In [ ]:
def powerfile(list2 = [], hotchannels = [], hotcores = []):
    line = []
    
    for i in range(4):
        if i in hotcores:
            line.append(str(0)+"\t")
        else:
            line.append(str(list2[i])+"\t")
    
    for i in range(16):
        if i in hotchannels:
            line.append(str(0)+"\t")
        else:
             line.append(list2[i]+"\t")
    for i in range(3):
        line.append(str(0)+"\t")
    for i in range(8):
        for k in range(16):
            if k in hotchannels:
                line.append(str(0)+"\t")
            else:
                line.append(list2[23+16*i+k]+"\t")
        for i in range(3):
            line.append(str(0)+"\t")
    return line

In [ ]:
head = []
for i in range(4):
    head.append("C_"+str(i)+"\t")
for i in range(16):
    head.append("LC_"+str(i)+"\t")
for i in range(3):
    head.append("X"+str(i+1)+"\t")
for i in range(8):
    for k in range(16):
        head.append("B_"+str(i*16+k)+"\t")
    for l in range(3):
        head.append("X"+str(l+1)+"\t")

In [ ]:
flag = [1]*16
stall = [0]*16
count = 0

In [ ]:
path = "/home/praveen/Downloads"

In [ ]:
ptrace = open(path+"/CoMeT-main/hotspot_tool3/oldarch/real_egnm.trace", 'r')
ptrace1 = ptrace.readlines()
length = len(ptrace1)-1
remaining = [999]*16
org = open(path+"/CoMeT-main/hotspot_tool3/oldarch/original_2.trace", 'w+')
org.writelines(head)
org.writelines("\n")
org.writelines(ptrace1[1])
org.writelines("\n")
org.close()

os.system("cd "+path+"/home/praveen/Downloads/CoMeT-main/hotspot_tool3/oldarch;"
          "../hotspot -c ../stack_hotspot.config -p original_2.trace -o temperature_mem_2.trace -model_secondary 1 -model_type grid -steady_file steady_temperature_mem_2.log -all_transient_file all_transient_mem_2.init -grid_steady_file grid_steady_mem_2.log -steady_state_print_disable 1 -l 1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1, -type 2.5D -sampling_intvl 0.001 -grid_layer_file ../stack.lcf -detailed_3D on -init_file ../stack.init")


temptrace = open(path+"/CoMeT-main/hotspot_tool3/oldarch/temperature_mem_2.trace", 'r')
content = temptrace.readlines()
temptrace.close()
line = content[1]

ttrace = open(path+"/CoMeT-main/hotspot_tool3/oldarch/t_thresh_egnm_77.trace", 'w')
ttrace.writelines(head)
ttrace.writelines("\n")
ttrace.writelines(line)

ff2 = open(path+"/CoMeT-main/hotspot_tool3/oldarch/power_thresh_egnm_77.trace", "w")
ff2.writelines(head)
ff2.writelines("\n")
ff2.writelines(ptrace1[1])

hb = open(path+"/CoMeT-main/hotspot_tool3/oldarch/hotblocks_thresh_egnm_77.txt", 'w')
line_no = 2
while((remaining[0]>0)|(remaining[1]>0)|(remaining[2]>0)|(remaining[3]>0)|(remaining[4]>0)|(remaining[5]>5)|(remaining[6]>0)|(remaining[7]>0)|(remaining[8]>0)|(remaining[9]>0)|(remaining[10]>0)|(remaining[11]>0)|(remaining[12]>0)|(remaining[13]>0)|(remaining[14]>0)|(remaining[15]>0)):
#     print("this is run number "+str(count+1))
    temptrace = open(path+"/CoMeT-main/hotspot_tool3/oldarch/temperature_mem_2.trace", 'r')
    #ttrace2.seek(0, os.SEEK_SET)
    content = temptrace.readlines()
    line2 = content[1].split()
    hotblocks = []
    hotcores = []
    for m in range(16):
        if((has_exceeded(m,77, line2)==1)|(remaining[m]<0)):
            hotblocks.append(m//4*4)
            hotblocks.append(m//4*4+1)
            hotblocks.append(m//4*4+2)
            hotblocks.append(m//4*4+3)
            hotcores.append(m//4)
#     print("the hotblocks are:\n")
#     print(hotblocks)
    hb.writelines("\nhotblocks in run number "+str(count)+" is: ")
    hb.writelines(str(hotblocks))
    
    if(len(hotblocks)==0):
        powertrace = [0]*175
        org = open(path+"/CoMeT-main/hotspot_tool3/oldarch/original_2.trace", 'w+')
        realtrace = numpy.array(pandas.read_csv(r path+"/CoMeT-main/hotspot_tool3/oldarch/real_egnm.trace",header = None, sep = "\t"))
        for cr in range(4):
            powertrace[cr] = str(realtrace[flag[4*cr]][cr])
        for lc in range(16):
            powertrace[lc+4] = "0.272"
        for x in range(20,23):
            powertrace[x] = "0"
        for layer in range(8):
            for chan in range(16):
                powertrace[23+layer*19+chan]  = str(realtrace[flag[chan]][23+layer*19+chan])
            for x in range(3):
                powertrace[23+layer*19+chan+x+1] = "0"
        org.writelines(head)
        org.writelines("\n")
        org.writelines("\t".join(powertrace))
        org.writelines("\n")
        org.close()
        
        os.system("cd "+path+"/home/praveen/Downloads/CoMeT-main/hotspot_tool3/oldarch;"
          "../hotspot -c ../stack_hotspot.config -p original_2.trace -o temperature_mem_2.trace -model_secondary 1 -model_type grid -steady_file steady_temperature_mem_2.log -all_transient_file all_transient_mem_2.init -grid_steady_file grid_steady_mem_2.log -steady_state_print_disable 1 -l 1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1, -type 2.5D -sampling_intvl 0.001 -grid_layer_file ../stack.lcf -detailed_3D on -init_file all_transient_mem_2.init")
        print("enter if block")
        ff2.writelines("\t".join(powertrace))
        ff2.writelines("\n")
        
    else:
        lv = []
        for f in range(16):
            if f in hotblocks:
                lv.append("0")
            else:
                lv.append("1")
        powertrace = ["0"]*175
        realtrace = numpy.array(pandas.read_csv(r path+"/CoMeT-main/hotspot_tool3/oldarch/real_egnm.trace",header = None, sep = "\t"))
        for cr in range(4):
            if cr in hotcores:
                powertrace[cr] = "0"
            else:
                powertrace[cr] = str(realtrace[flag[4*cr]][cr])
        for lc in range(16):
            if lc in hotblocks:
                powertrace[lc+4] = "0"
            else:
                powertrace[lc+4] = "0.272"
        for x in range(20,23):
            powertrace[x] = "0"
        for layer in range(8):
            for chan in range(16):
                if x in hotblocks:
                    powertrace[23+layer*19+chan] = "0"
                else:
                    powertrace[23+layer*19+chan]  = str(realtrace[flag[chan]][23+layer*19+chan])
            for x in range(3):
                powertrace[23+layer*19+chan+x+1] = "0"
        ff = open(path+"/CoMeT-main/hotspot_tool3/oldarch/new_2.trace", "w+")
        print("enter else block")
        ff.writelines(head)
        ff.writelines("\n")
        ff.writelines("\t".join(powertrace))
        ff.writelines("\n")
        
        ff2.writelines("\t".join(powertrace))
        ff2.writelines("\n")
        ff.close()
        os.system("cd "+path+"/home/praveen/Downloads/CoMeT-main/hotspot_tool3/oldarch;"
          "../hotspot -c ../stack_hotspot.config -p new_2.trace -o temperature_mem_2.trace -model_secondary 1 -model_type grid -steady_file steady_temperature_mem_2.log -all_transient_file all_transient_mem_2.init -grid_steady_file grid_steady_mem_2.log -steady_state_print_disable 1 -l "+",".join(lv)+","+" -type 2.5D -sampling_intvl 0.001 -grid_layer_file ../stack.lcf -detailed_3D on -init_file all_transient_mem_2.init")
        
       
        
    temptrace = open(path+"/CoMeT-main/hotspot_tool3/oldarch/temperature_mem_2.trace", 'r')
    content = temptrace.readlines()
    line = content[1]
    temptrace.close()
    ttrace.writelines(line)
    
    for k in range(16):
        if k not in hotblocks:
            remaining[k] = remaining[k]-1
            flag[k] = flag[k]+1
        else:
            stall[k] = stall[k]+1
    print("remaining :", remaining)
#     print("stall", stall)
#     print("flag ", flag)
    count = count+1
#     print("count:", count)
    
    line_no = line_no+1
    
ttrace.close()
ff2.close()
hb.close()

In [ ]:
import time
print(time.strftime("%H:%M:%S"))           ##start at 12:29